In [ ]:
#handle warnings
import warnings
warnings.filterwarnings("ignore")

import shapely                 #needed to set geopandas geometry 
from shapely.wkt import loads  #needed to set geopandas geometry

import pandas as pd            #for DataFrame and analysis
import geopandas as gpd        #for mapping and spatial aggregation
import mapclassify as mp   #to view quantitative classification summaries

import numpy as np        #for working with arrays from mapclassify

import matplotlib.pyplot as plt  #use matplotlib to modify the plots (e.g. make them larger) 
import palettable.colorbrewer.sequential as pcs  #for colorBrewer palettes



In [ ]:
df_crime = pd.read_csv("MPS LSOA Level Crime (Historical).csv")
lsoa_gdf = gpd.read_file("LSOA_2011_London_gen_MHW.shp")

In [ ]:
months_2019 = [f"2019{str(m).zfill(2)}" for m in range(1, 13)]
crime_2019 = df_crime[['LSOA Code'] + months_2019]

crime_2019_lsoa = crime_2019[crime_2019['LSOA Code'].str.startswith('E01')]

# Step 2: Filter and group by LSOA Code
crime_2019_lsoa = (
    df_crime[['LSOA Code'] + months_2019]
    .groupby('LSOA Code')
    .sum()
)

crime_2019_lsoa['Total_2019_Crime'] = crime_2019_lsoa.sum(axis=1)

crime_2019_lsoa = crime_2019_lsoa.reset_index()

# Step 5: Merge with shapefile population data using LSOA code
lsoa_pop = lsoa_gdf[['LSOA11CD', 'USUALRES']].copy()
lsoa_pop = lsoa_pop.rename(columns={'LSOA11CD': 'LSOA Code'})

crime_2019_lsoa = crime_2019_lsoa.merge(lsoa_pop, on='LSOA Code', how='left')

# Step 6: Calculate crime rate per 1,000 residents
crime_2019_lsoa['Crime_Rate_per_1000'] = (
    crime_2019_lsoa['Total_2019_Crime'] / crime_2019_lsoa['USUALRES']
) * 1000


In [ ]:
crime_2019_lsoa.head()

In [ ]:
merged_crime_gdf = lsoa_gdf.merge(
    crime_2019_lsoa,
    left_on='LSOA11CD',
    right_on='LSOA Code',
    how='left'
)


In [ ]:
print(merged_crime_gdf[['LSOA11CD', 'Total_2019_Crime', 'USUALRES_y', 'Crime_Rate_per_1000']].dropna().head())

In [ ]:
merged_crime_gdf.head()

In [ ]:
merged_crime_gdf = merged_crime_gdf.dropna(subset=['Total_2019_Crime'])

In [ ]:
merged_crime_gdf.head()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

merged_crime_gdf.plot(
    column='Crime_Rate_per_1000',
    cmap='Reds',
    linewidth=0.1,
    edgecolor='grey',
    legend=True,
    ax=ax,
    missing_kwds={'color': 'lightgrey', "label": "No Data"}
)

ax.set_title("Total Crime per LSOA in London (2019)", fontsize=15)
ax.set_axis_off()
plt.show()

In [ ]:
print(merged_crime_gdf['LSOA Code'].value_counts().head())

In [ ]:
print(merged_crime_gdf['Crime_Rate_per_1000'].max())

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(12, 10))   
merged_crime_gdf.plot(column='Crime_Rate_per_1000', ax=ax1,
                      scheme='natural_breaks',
         cmap='viridis',
         edgecolor='grey', linewidth=0.1,        #change line style
         legend=True, legend_kwds={'title': "Number of Crimes",'loc': 'lower right'})  

ax1.axis('off') #don't plot the axes (bounding box)

plt.title('Crime Rate per 1000 people by LSOA, 2019', fontsize=20)  #provide a title

ax1.annotate(xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')  #add source info on the image itself

plt.show()

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import colors 
# Analysis
import geopandas as gpd 
import pandas as pd
from libpysal import weights 
from pysal.explore import esda
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import shapely                 #needed to set geopandas geometry 
from shapely.wkt import loads  #needed to set geopandas geometry

import pandas as pd            #for DataFrame and analysis
import geopandas as gpd        #for mapping and spatial aggregation
import mapclassify as mp   #to view quantitative classification summaries

import numpy as np        #for working with arrays from mapclassify

import matplotlib.pyplot as plt  #use matplotlib to modify the plots (e.g. make them larger) 
import palettable.colorbrewer.sequential as pcs  #for colorBrewer palettes

In [ ]:
w = weights.KNN.from_dataframe(merged_crime_gdf, k=8)
w.transform = 'R'

In [ ]:
merged_crime_gdf['Crime_Rate_log'] = np.log1p(merged_crime_gdf['Crime_Rate_per_1000'])
moran = esda.moran.Moran(merged_crime_gdf['Crime_Rate_log'], w)

In [ ]:
round(moran.I,3)

In [ ]:
moran.p_sim

In [ ]:
plt.hist(moran.sim, 10, facecolor='lightblue', edgecolor='black')
plt.vlines(moran.I, 0, 350, color='r', linestyle="--")
plt.vlines(moran.EI, 0, 350)
plt.xlabel("Moran's I")
plt.ylabel("Count")
plt.show()

In [ ]:
merged_crime_gdf['Crime_Rate_Log_lag'] = weights.spatial_lag.lag_spatial(w, merged_crime_gdf['Crime_Rate_log'])

In [ ]:
def standardize(df, var):
    newname = var + '_z'
    df[newname] = (df[var] - df[var].mean()) / df[var].std()

standardize(merged_crime_gdf,'Crime_Rate_log')
standardize(merged_crime_gdf, 'Crime_Rate_Log_lag')


In [ ]:
merged_crime_gdf.tail()

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,10))

ax=ax.flatten()


for i, p in enumerate(['Crime_Rate_log',
                       'Crime_Rate_log_z','Crime_Rate_Log_lag','Crime_Rate_Log_lag_z']):
    
    merged_crime_gdf.plot(column=p, cmap='viridis', 
                   scheme='quantiles', k=5,
                   linewidth=0.,
                   legend=True, legend_kwds={"title":p, "loc": 3}, ax=ax[i]
                  )
    ax[i].set_axis_off()
    
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))


plt.scatter(merged_crime_gdf['Crime_Rate_log_z'], merged_crime_gdf['Crime_Rate_Log_lag_z'], color='slateblue', alpha=0.7)

# Add reference lines at 0
plt.axhline(0, color='grey', linestyle='--')
plt.axvline(0, color='grey', linestyle='--')

# Labels and title
ax.set_title("Moran Scatterplot of Number of Crimes Recorded in London by LSOA(2019)", fontsize=13)
ax.set_xlabel("Standardised Crime Rate (z-score)", fontsize=11)
ax.set_ylabel("Spatial Lag of Crime Rate (z-score)", fontsize=11)


plt.tight_layout()
plt.show()

In [ ]:
merged_crime_gdf['Crime_Rate_per_1000'] = (
    merged_crime_gdf['Total_2019_Crime'] / merged_crime_gdf['USUALRES']
) * 1000


In [ ]:
merged_crime_gdf.head()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))

plt.scatter(
    merged_crime_gdf['Crime_Rate_log_z'], 
    merged_crime_gdf['Crime_Rate_Log_lag_z'], 
    alpha=0.6, 
    color='slateblue'
)


ax.axvline(0, color='k', linestyle='--', alpha=0.6)
ax.axhline(0, color='k', linestyle='--', alpha=0.6)


plt.text(1, 1.5, "HH", fontsize=20, color='darkred')
plt.text(1, -1.5, "HL", fontsize=20, color='darkorange')
plt.text(-1.75, 1.5, "LH", fontsize=20, color='darkgreen')
plt.text(-1.75, -1.5, "LL", fontsize=20, color='navy')

ax.set_title("Moran Scatterplot of Log-Transformed Crime Rate (London, 2019)", fontsize=13)
ax.set_xlabel("Standardised Log Crime Rate (z-score)", fontsize=11)
ax.set_ylabel("Spatial Lag of Log Crime Rate (z-score)", fontsize=11)


plt.tight_layout()
plt.show()

In [ ]:
def assign_quadrant(row):
    if row['Crime_Rate_log_z'] > 0:
        if row['Crime_Rate_Log_lag_z'] > 0:
            return 'HH'  
        else:
            return 'HL'  
    else:
        if row['Crime_Rate_Log_lag_z'] > 0:
            return 'LH'  
        else:
            return 'LL'  

merged_crime_gdf['quadrant'] = merged_crime_gdf.apply(assign_quadrant, axis=1)

In [ ]:
qcolors = {'HH':'red', 'HL':'pink', 'LH':'lightblue', 'LL':'blue'}
cmap = colors.ListedColormap(qcolors.values())

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

merged_crime_gdf.plot(
    column='quadrant',
    categorical=True,
    cmap=cmap,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Moran I Quadrant", "loc": 3},
    ax=ax[0]
)
ax[0].set_title("Local Spatial Clusters (IMD Rank)")
ax[0].axis("off")

ax[1].scatter(
    merged_crime_gdf['Crime_Rate_log_z'],
    merged_crime_gdf['Crime_Rate_Log_lag_z'],
    c=merged_crime_gdf['quadrant'].map(qcolors),
    alpha=0.7
)
ax[1].axvline(0, color='k', linestyle='--', alpha=0.5)
ax[1].axhline(0, color='k', linestyle='--', alpha=0.5)

ax[1].set_xlabel("Crime Rate (z)")
ax[1].set_ylabel("Spatial Lag of Crime Rate (z)")
ax[1].set_title("Moran Scatterplot")

plt.tight_layout()
plt.show()

In [ ]:
lisa = esda.moran.Moran_Local(merged_crime_gdf['Crime_Rate_log'], w)

In [ ]:
merged_crime_gdf['Is'] = lisa.Is

fig, ax = plt.subplots(1, figsize=(9, 9))
merged_crime_gdf.plot(column='Is', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Local Moran's I","loc": 2}, ax=ax
       )
ax.set_axis_off()

In [ ]:
plt.hist(lisa.Is, 10, facecolor='lightblue', edgecolor='black')
plt.xlabel("Local Moran's I")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.hist(lisa.p_sim, 10, facecolor='lightblue', edgecolor='black')
plt.xlabel("Local Moran's I p-value")
plt.ylabel("Count")
plt.show()

In [ ]:
merged_crime_gdf['p-sim'] = lisa.p_sim
sig = 1 * (lisa.p_sim < 0.05) 
slabels = ['non-sig.', 'significant'] 
labels = [slabels[i] for i in sig]
merged_crime_gdf['sig'] = labels
merged_crime_gdf[['sig','p-sim']].head(100)

In [ ]:
fig, ax = plt.subplots(1,figsize=(10,8))
sigcolors = {'non-sig.':'lightgrey', 'significant':'black'}
merged_crime_gdf.plot(column='sig', categorical=True, cmap=colors.ListedColormap(sigcolors. 
    values()),
            k=2, linewidth=0.1, edgecolor='white',
               legend=True, legend_kwds={"title":'Local Moran I',"loc": 2}, ax=ax)
plt.show()

In [ ]:
merged_crime_gdf['sigIs'] = np.where(merged_crime_gdf['sig'] == 'significant', merged_crime_gdf['Is'], np.nan)

fig, ax = plt.subplots(1, figsize=(10, 8))

merged_crime_gdf.plot(color='lightgrey', linewidth=0.0, ax=ax)

merged_crime_gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.5,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": "upper left"},
    ax=ax
)

ax.set_title("Statistically Significant Local Moran's I – Crime Rate", fontsize=14)
ax.set_axis_off()
plt.show()

In [ ]:
merged_crime_gdf['sigIs'] = np.where(merged_crime_gdf['sig'] == 'significant', merged_crime_gdf['Is'], np.nan)


fig, ax = plt.subplots(1, figsize=(10, 8))

merged_crime_gdf.plot(color='lightgrey', linewidth=0.2, ax=ax)

merged_crime_gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": 2},
    ax=ax
)

ax.set_title("Significant Local Moran's I for Crime Rate (2019)", fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
qcolors = {'HH': 'red', 'HL': 'pink', 'LL': 'lightblue', 'LH': 'blue'}
cmap_quadrant = colors.ListedColormap(qcolors.values())


fig, ax = plt.subplots(1, 2, figsize=(12, 6))

merged_crime_gdf.plot(
    column='quadrant',
    categorical=True,
    cmap=cmap_quadrant,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": 'Moran I Quadrant', "loc": 3},
    ax=ax[0]
)
ax[0].set_title("Moran's I Clusters (Crime Rate)")
ax[0].axis('off')

merged_crime_gdf.plot(
    color='lightgrey',
    linewidth=0.2,
    ax=ax[1]
)
merged_crime_gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": 3},
    ax=ax[1]
)
ax[1].set_title("Significant Local Moran's I (Crime Rate)")
ax[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
q_labels = ['HH', 'LH', 'LL', 'HL']  

labels = [q_labels[i - 1] for i in lisa.q]

merged_crime_gdf['quadrant'] = labels

sig = lisa.p_sim < 0.05

hotspot   = 1 * (sig * (lisa.q == 1))  # HH
doughnut  = 2 * (sig * (lisa.q == 2))  # LH
coldspot  = 3 * (sig * (lisa.q == 3))  # LL
diamond   = 4 * (sig * (lisa.q == 4))  # HL

spots = hotspot + doughnut + coldspot + diamond

spot_labels = ['0 non-sig.', '1 hot spot', '2 doughnut', '3 cold spot', '4 diamond']
labels = [spot_labels[i] for i in spots]

merged_crime_gdf['slabels'] = labels

fig, ax = plt.subplots(1, figsize=(12, 12))

sig_colors = colors.ListedColormap(['lightgrey', 'red', 'lightblue', 'blue', 'pink'])

merged_crime_gdf.plot(
    column='slabels',
    categorical=True,
    cmap=sig_colors,
    k=2,
    linewidth=0.1,
    edgecolor='white',
    legend=True,
    legend_kwds={'title': "LISA (Moran's I)", 'loc': 2},
    ax=ax
)

ax.set_axis_off()
ax.set_title("LISA Cluster Map (Log Crime Rate per 1,000 London LSOAs (2019)", fontsize=15)
plt.show()